In [2]:
#Necessary packages
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from math import sqrt
import datetime 
import calendar
import sys

from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import VECM, coint_johansen

In [3]:
# Load the data from the Excel file
file_path = r'C:\Users\PHIRI003\OneDrive - Wageningen University & Research\Documents\WEcR Internship\Work\Dairy\Merged_data.xlsx'
df = pd.read_excel(file_path, header=0 )

df['Date'] = pd.to_datetime(df['Date'])  # Ensure 'Date' is in datetime format
df.set_index('Date', inplace=True)

In [1]:
#print(df.head())

In [4]:
# Define the function to create seasonal dummy variables
def SeasonalDummies(df, frequency='M'):
    nT = len(df)  # Number of observations
    startdate = df.index[0]  # Start date of the time series
    datetime = pd.DataFrame(data=pd.date_range(startdate, periods=nT, freq=frequency), columns=["datetime"])
    monthnumber = datetime["datetime"].dt.month  # Extract month numbers
    monthname = pd.DataFrame()  # DataFrame to store month names
    
    for i in range(nT):
        monthname.at[i, 'D'] = calendar.month_name[monthnumber[i]]  # Assign month names based on month numbers
    
    seasdum = pd.get_dummies(monthname)  # Create dummy variables for each month
    seasdum = seasdum.drop('D_January', axis=1)  # Drop January to avoid multicollinearity
    seasdum = seasdum.set_index(df.index)  # Set the index to match the original DataFrame
    seasdum = seasdum.astype(int)
    return seasdum

# Create seasonal dummies
seasdum = SeasonalDummies(df)

# Combine the data with the seasonal dummy variables
df = pd.concat([df, seasdum], axis=1)

#print(df.head()) 

In [5]:
#Create differenced exogenous lags
columns = ['API', 'PPI', 'CPI', 'Oil', 'Silage_PI', 'Dairy_Conce', 'CWage_20']

for column in columns:
    df[f'diff_{column}'] = df[column].diff()


In [8]:
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# Define break dates
break_dates = [pd.Timestamp('2015-01-01'), pd.Timestamp('2021-09-01')]

#create a trend variable
df['trend'] = range(1, len(df) + 1)  

# Variables to test and exogenous variables
vars_to_test = ['API', 'PPI', 'CPI']  # Replace with your actual variable names
exog = ['D_April', 'D_August', 'D_December', 'D_February', 'D_July', 'D_June', 'D_March', 'D_May', 'D_September', 'D_October', 'D_November']  

# Split the DataFrame into segments based on break dates
segments = []
start_date = df.index[0]

for break_date in break_dates:
    segments.append(df.loc[start_date:break_date])
    start_date = break_date + pd.Timedelta(days=1)

# Append the last segment
segments.append(df.loc[start_date:])

# Estimate VAR models and determine cointegration rank for each segment
optimal_lag = 4  
var_models = []
cointegration_ranks = []
johansen_tests = []  # List to store Johansen test results

for i, segment in enumerate(segments):
    segment_endog = segment[vars_to_test]  # endogenous variables
    segment_exog = segment[exog]  # Include dummy variables and additional exogenous variables
    
    # Fit the VECM model with exogenous variables
    model = VAR(segment_endog, exog=segment_exog)
    fitted_model = model.fit(4)
    var_models.append((fitted_model, segment_exog))

    # Perform Johansen cointegration test to determine the rank
    johansen_test = coint_johansen(segment_endog.values, det_order=1, k_ar_diff=optimal_lag)
    johansen_tests.append(johansen_test)
    
    rank = 0
    for j in range(len(johansen_test.lr1)):
        if johansen_test.lr1[j] > johansen_test.cvt[j, 1]:  # Compare with 95% critical value
            rank = j + 1
        else:
            break
    cointegration_ranks.append(rank)

# Compile and display the results of the Johansen cointegration test
for i, test in enumerate(johansen_tests):
    print(f"Segment {i+1}")
    print("Trace Statistic:")
    print(test.lr1)  # Trace statistic
    print("Critical Values (90%, 95%, 99%):")
    print(test.cvt)  # Critical values

    rank = 0
    for j in range(len(test.lr1)):
        if test.lr1[j] > test.cvt[j, 1]:  # Compare with 95% critical value
            rank = j + 1
        else:
            break
    print(f"Rank: {rank}")
    print()


Segment 1
Trace Statistic:
[36.0663325  16.40359584  6.46905329]
Critical Values (90%, 95%, 99%):
[[32.0645 35.0116 41.0815]
 [16.1619 18.3985 23.1485]
 [ 2.7055  3.8415  6.6349]]
Rank: 1

Segment 2
Trace Statistic:
[37.22353001 12.63415677  5.19579665]
Critical Values (90%, 95%, 99%):
[[32.0645 35.0116 41.0815]
 [16.1619 18.3985 23.1485]
 [ 2.7055  3.8415  6.6349]]
Rank: 1

Segment 3
Trace Statistic:
[107.79682749  35.01268619   2.50536228]
Critical Values (90%, 95%, 99%):
[[32.0645 35.0116 41.0815]
 [16.1619 18.3985 23.1485]
 [ 2.7055  3.8415  6.6349]]
Rank: 2



C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [7]:
##Estimating segrement by segment
# Define egment 1
start_date_seg_1 = '2010-01-01'
end_date_seg_1 = '2015-01-01'
segment_1 = df.loc[start_date_seg_1:end_date_seg_1]

Y1 = segment_1[['API', 'PPI', 'CPI']]
X1 =  segment_1[['trend', 'diff_Oil', 'diff_Silage_PI', 'diff_Dairy_Conce', 'diff_CWage_20', 'D_April', 'D_August', 'D_December', 'D_February', 'D_July', 'D_June', 'D_March', 'D_May', 'D_September', 'D_October', 'D_November']]
X1 = sm.add_constant(X1)


X1 = X1.replace([np.inf, -np.inf], np.nan).dropna()
aligned_index = X1.index.intersection(Y1.index)
Y1 = Y1.loc[aligned_index]
X1 = X1.loc[aligned_index]

vecm_1 = VECM(endog=Y1, exog=X1, k_ar_diff=4, coint_rank=1)  # defining the equation
vecm_fit_1 = vecm_1.fit()
print(vecm_fit_1.summary())


Det. terms outside the coint. relation & lagged endog. parameters for equation API
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
exog1        162.6657     53.739      3.027      0.002      57.339     267.992
exog2          0.2513      0.094      2.669      0.008       0.067       0.436
exog3          0.0592      0.035      1.686      0.092      -0.010       0.128
exog4         -0.2884      0.130     -2.220      0.026      -0.543      -0.034
exog5         -0.9707      0.294     -3.307      0.001      -1.546      -0.395
exog6         -6.3753      3.828     -1.665      0.096     -13.878       1.128
exog7         -1.6370      3.141     -0.521      0.602      -7.793       4.519
exog8          6.6293      3.760      1.763      0.078      -0.740      13.999
exog9        -11.3358      3.806     -2.979      0.003     -18.795      -3.877
exog10        -4.1545      3.556     -1.168     

C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [10]:
# Define egment 2
start_date_seg_2 = '2015-02-01'
end_date_seg_2 = '2021-09-01'
segment_2 = df.loc[start_date_seg_2:end_date_seg_2]

#Y2 = segment_2[['diff_API', 'diff_PPI', 'diff_CPI']]
Y2 = segment_2[['API', 'PPI', 'CPI']]
X2 =  segment_2[['trend', 'diff_Oil', 'diff_Silage_PI', 'diff_Dairy_Conce', 'diff_CWage_20', 'D_April', 'D_August', 'D_December', 'D_February', 'D_July', 'D_June', 'D_March', 'D_May', 'D_September', 'D_October', 'D_November']]
#X2 = sm.add_constant(X2)

X2 = X2.replace([np.inf, -np.inf], np.nan).dropna()
aligned_index = X2.index.intersection(Y2.index)
Y2 = Y2.loc[aligned_index]
X2 = X2.loc[aligned_index]

vecm_2 = VECM(endog=Y2, exog=X2, k_ar_diff=4, coint_rank=1)  # defining the equation
#var = VAR(endog=Y2, exog=X2)
vecm_fit_2 = vecm_2.fit()
#var_fit= var.fit(4)
#print(var_fit.summary())
print(vecm_fit_2.summary())

Det. terms outside the coint. relation & lagged endog. parameters for equation API
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
exog1          0.0004      0.023      0.019      0.985      -0.045       0.046
exog2         -0.0103      0.030     -0.339      0.735      -0.070       0.049
exog3         -0.0730      0.136     -0.539      0.590      -0.339       0.193
exog4          1.5350      0.532      2.883      0.004       0.491       2.579
exog5         -0.5082      2.141     -0.237      0.812      -4.704       3.688
exog6          5.6555      3.483      1.624      0.104      -1.172      12.483
exog7         12.0102      3.427      3.505      0.000       5.294      18.727
exog8          8.0961      2.947      2.747      0.006       2.320      13.872
exog9          9.9227      2.785      3.563      0.000       4.465      15.381
exog10        11.6622      2.418      4.822     

C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [11]:
##Limited observation causing estimation problems
# Define segment 3
start_date_seg_3 = '2021-10-01'
end_date_seg_3 = '2023-10-01'
segment_3 = df.loc[start_date_seg_3:end_date_seg_3]

Y3 = segment_3[['API', 'PPI', 'CPI']]
X3 =  segment_3[['diff_Oil', 'diff_Silage_PI', 'diff_Dairy_Conce', 'diff_CWage_20', 'D_April', 'D_August', 'D_December', 'D_February', 'D_July', 'D_June', 'D_March', 'D_May', 'D_September', 'D_October', 'D_November']]
#X3 = sm.add_constant(X3)

X3 = X3.replace([np.inf, -np.inf], np.nan).dropna()
aligned_index = X3.index.intersection(Y3.index)
Y3 = Y3.loc[aligned_index]
X3 = X3.loc[aligned_index]

vecm_3 = VECM(endog=Y3, exog=X3, k_ar_diff=4, coint_rank=2)  # defining the equation
vecm_fit_3 = vecm_3.fit()
print(vecm_fit_3.summary())

Det. terms outside the coint. relation & lagged endog. parameters for equation API
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
exog1          0.7348   3.01e+07   2.44e-08      1.000    -5.9e+07     5.9e+07
exog2          5.4733        nan        nan        nan         nan         nan
exog3         -1.5260        nan        nan        nan         nan         nan
exog4          8.1826        nan        nan        nan         nan         nan
exog5         13.6002        nan        nan        nan         nan         nan
exog6        -17.4020   2.93e+08  -5.93e-08      1.000   -5.75e+08    5.75e+08
exog7       -205.9658        nan        nan        nan         nan         nan
exog8        -73.6507        nan        nan        nan         nan         nan
exog9        -62.1514   1.98e+09  -3.14e-08      1.000   -3.88e+09    3.88e+09
exog10      -219.1397        nan        nan     

C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\vector_ar\vecm.py:1547: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params_default))
